<a href="https://colab.research.google.com/github/sammysingle/Thesis/blob/main/ColabWorkspaces/thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VGG-16

*Original Dataset*

In [ ]:
# Import os library to navigate directories
import os
# Import tensorflow functions for model building
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Input, GlobalAveragePooling2D
from tensorflow.keras import Model
from tensorflow.keras.models import save_model
# Import tensorflow functions for training
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.data import AUTOTUNE as autotune
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import cast
# Import tensorflow functions for metrics
from tensorflow.keras.metrics import Accuracy, Precision, Recall

# ------------------------------ Build VGG-16 ------------------------------ #
# Create base VGG-16 model:
  # 224x224x3 input size
  # Only feature extraction layers - no classification layers
  # Feature extraction layer weights trained on ImageNet dataset
base_model = VGG16(input_shape = (224, 224, 3),
                         include_top = False,
                         weights = 'imagenet'
                         )
# Freeze learning on feature extraction layers
base_model.trainable = False

# Create classification layers of VGG16
# Global average pooling
global_average_pooling = GlobalAveragePooling2D()
# Fully connected layer 1: 4096 neurons ReLU activation function
fc1 = Dense(units = 4096, activation = 'relu')
# Fully connected layer 2: 4096 neurons ReLU activation function
fc2 = Dense(units = 4096, activation = 'relu')
# Classification layer: softmax layer with 8 neurons for class labels
classifier = Dense(units = 8, activation = 'softmax')

# Connect input layer, base model and output layers
inputs = Input(shape = (224, 224, 3)) # Input layer: 224x224 RGB images
model = base_model(inputs)            # Feature extraction layers of VGG-16
model = global_average_pooling(model) # Global average pooling layer
model = fc1(model)                    # Fully connected layer: 4096 neuron ReLU
model = fc2(model)                    # Fully connected layer: 4096 neuron ReLU
outputs = classifier(model)           # Fully connected layer: 8 neuron softmax activation for classification

# Build VGG-16 model
vgg16 = Model(inputs, outputs, name = 'VGG-16')

# ------------------------------ Train VGG-16 ------------------------------ #
# RGB value normalization to improve learning rate - RGB values within activation function range
def process(image, label):
    image = cast(image/255.0, 'float32')
    return image, label

# Define training parameters
batch_size = 4   # Batch size for gradient learning
img_height = 224 # Image pixel height
img_width = 224  # Image pixel width
epochs = 200     # Arbitrarily large epoch for training

# Define training and validation directory paths
train_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Original Training Dataset'
validation_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Validation Dataset'

# Create training dataset
train_ds = image_dataset_from_directory(
    train_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize training dataset pixel values to improve learning performance
train_ds = train_ds.map(process)
# Prefetch training dataset to improve computational speed
train_ds = train_ds.cache().prefetch(buffer_size = autotune)

# Define validation dataset
val_ds = image_dataset_from_directory(
    validation_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize validation pixel values to improve learning performance
val_ds = val_ds.map(process)
# Prefetch validation dataset to improve computation speed
val_ds = val_ds.cache().prefetch(buffer_size = autotune)

# Compile model:
  # Adam algorithm optimization
  # Categorical crossentropy loss function for multilabel classification
  # Metrics: accuracy, precision, recall
vgg16.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy', Precision(), Recall()]
)

# Define checkpoint to save model parameters to epoch with best performance
model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Models/VGG-16/Original Training/Checkpoint', monitor = 'val_accuracy', mode = 'max')

# Train model to run until epochs finished or callback threshold reached
vgg16.fit(
    train_ds,
    validation_data = val_ds,
    epochs = epochs,
    verbose = 1,
    callbacks = [model_checkpoint]
)

# Save VGG-16 model at end of training
save_model(model = vgg16, 
           filepath = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Models/VGG-16/Original Training/End',
           include_optimizer = True,
)

# ------------------------------ Test VGG-16 ------------------------------ #
# Define testing dataset
print('\n')
test_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Testing Dataset'
test_ds = image_dataset_from_directory(
    test_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = 1,
    label_mode = 'categorical'
)
# Normalize testing pixel values
test_ds = test_ds.map(process)

# Perform predictions on testing dataset to obtain accuracy, precision and recall
loss, accuracy, precision, recall = vgg16.evaluate(test_ds)
print('\nTest Accuracy: ' + str(accuracy))
print('Test Precision: ' + str(precision))
print('Test Recall: ' + str(recall))

*Augmented Dataset*

In [11]:
# Import os library to navigate directories
import os
# Import tensorflow functions for model building
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import Model
from tensorflow.keras.models import save_model
# Import tensorflow functions for training
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.data import AUTOTUNE as autotune
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import cast
# Import tensorflow functions for metrics
from tensorflow.keras.metrics import Accuracy, Precision, Recall

# ------------------------------ Build VGG-16 ------------------------------ #
# Create base VGG-16 model:
  # 224x224x3 input size
  # Only feature extraction layers - no classification layers
  # Feature extraction layer weights trained on ImageNet dataset
base_model = VGG16(input_shape = (224, 224, 3),
                         include_top = False,
                         weights = 'imagenet'
                         )
# Freeze learning on feature extraction layers
base_model.trainable = False

# Create classification layers of VGG16
# Global average pooling
global_average_pooling = GlobalAveragePooling2D()
# Fully connected layer 1: 4096 neurons ReLU activation function
fc1 = Dense(units = 4096, activation = 'relu')
# Fully connected layer 2: 4096 neurons ReLU activation function
fc2 = Dense(units = 4096, activation = 'relu')
# Classification layer: softmax layer with 8 neurons for class labels
classifier = Dense(units = 8, activation = 'softmax')

# Connect input layer, base model and output layers
inputs = Input(shape = (224, 224, 3)) # Input layer: 224x224 RGB images
model = base_model(inputs)            # Feature extraction layers of VGG-16
model = global_average_pooling(model) # Global average pooling layer
model = fc1(model)                    # Fully connected layer: 4096 neuron ReLU
model = fc2(model)                    # Fully connected layer: 4096 neuron ReLU
outputs = classifier(model)           # Fully connected layer: 8 neuron softmax activation for classification

# Build VGG-16 model
vgg16 = Model(inputs, outputs, name = 'VGG-16')

# ------------------------------ Train VGG-16 ------------------------------ #
# RGB value normalization to improve learning rate - RGB values within activation function range
def process(image, label):
    image = cast(image/255.0, 'float32')
    return image, label

# Define training parameters
batch_size = 4   # Batch size for gradient learning
img_height = 224 # Image pixel height
img_width = 224  # Image pixel width
epochs = 200     # Arbitrarily large epoch for training

# Define training and validation directory paths
train_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Augmented Training Dataset'
validation_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Validation Dataset'

# Create training dataset
train_ds = image_dataset_from_directory(
    train_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize training dataset pixel values to improve learning performance
train_ds = train_ds.map(process)
# Prefetch training dataset to improve computational speed
train_ds = train_ds.cache().prefetch(buffer_size = autotune)

# Define validation dataset
val_ds = image_dataset_from_directory(
    validation_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize validation pixel values to improve learning performance
val_ds = val_ds.map(process)
# Prefetch validation dataset to improve computation speed
val_ds = val_ds.cache().prefetch(buffer_size = autotune)

# Compile model:
  # Adam algorithm optimization
  # Categorical crossentropy loss function for multilabel classification
  # Metrics: accuracy, precision, recall
vgg16.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy', Precision(), Recall()]
)

# Define checkpoint to save model parameters to epoch with best performance
model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Models/VGG-16/Augmented Training/Checkpoint', monitor = 'val_accuracy', mode = 'max')

# Train model to run until epochs finished or callback threshold reached
vgg16.fit(
    train_ds,
    validation_data = val_ds,
    epochs = epochs,
    verbose = 1,
    callbacks = [model_checkpoint]
)

# Save VGG-16 model at end of training
save_model(model = vgg16, 
           filepath = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Models/VGG-16/Augmented Training/End',
           include_optimizer = True,
)

# ------------------------------ Test VGG-16 ------------------------------ #
# Define testing dataset
print('\n')
test_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Testing Dataset'
test_ds = image_dataset_from_directory(
    test_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = 1,
    label_mode = 'categorical'
)
# Normalize testing pixel values
test_ds = test_ds.map(process)

# Perform predictions on testing dataset to obtain accuracy, precision and recall
loss, accuracy, precision, recall = vgg16.evaluate(test_ds)
print('\nTest Accuracy: ' + str(accuracy))
print('Test Precision: ' + str(precision))
print('Test Recall: ' + str(recall))

Found 10083 files belonging to 8 classes.
Found 695 files belonging to 8 classes.
Epoch 1/200
   7/2521 [..............................] - ETA: 1:17:28 - loss: 3.6375 - accuracy: 0.1071 - precision_3: 0.1250 - recall_3: 0.0357

KeyboardInterrupt: ignored

# ResNet-50

*Original Dataset*

In [ ]:
# Import os library to navigate directories
import os
# Import tensorflow functions for model building
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, Input, GlobalAveragePooling2D
from tensorflow.keras import Model
from tensorflow.keras.models import save_model
# Import tensorflow functions for training
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.data import AUTOTUNE as autotune
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import cast
# Import tensorflow functions for metrics
from tensorflow.keras.metrics import Accuracy, Precision, Recall

# ----------------------------- Build ResNet-50 ---------------------------- #
# Create base ResNet-50 model:
  # 224x224x3 input size
  # Only feature extraction layers - no classification layers
  # Feature extraction layer weights trained on ImageNet dataset
base_model = ResNet50(input_shape = (224, 224, 3),
                         include_top = False,
                         weights = 'imagenet'
                         )
# Freeze learning on feature extraction layers
base_model.trainable = False

# Create classification layers of ResNet-50
global_average_pool = GlobalAveragePooling2D()        # Global average pooling
fc1 = Dense(units = 1000, activation = 'relu')        # 1000 neuron fully connected layer with ReLU activation
classifier = Dense(units = 8, activation = 'softmax') # Softmax classifier

# Connect input layer, base model and output layers
inputs = Input(shape = (224, 224, 3)) # Input layer: 224x224 RGB images
model = base_model(inputs)            # Feature extraction layers of ResNet-50
model = global_average_pool(model)    # Global average pooling layer
model = fc1(model)                    # Fully connected layer: 1000 neuron ReLU
outputs = classifier(model)           # Fully connected layer: 8 neuron softmax activation for classification

# Build res_net_50 model
res_net_50 = Model(inputs, outputs, name = 'ResNet-50')

# ----------------------------- Train ResNet-50 ---------------------------- #
# RGB value normalization to improve learning rate - RGB values within activation function range
def process(image, label):
    image = cast(image/255.0, 'float32')
    return image, label

# Define training parameters
batch_size = 4   # Batch size for gradient learning
img_height = 224 # Image pixel height
img_width = 224  # Image pixel width
epochs = 200     # Arbitrarily large epoch for training

# Define training and validation directory paths
train_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Original Training Dataset'
validation_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Validation Dataset'

# Create training dataset
train_ds = image_dataset_from_directory(
    train_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize training dataset pixel values to improve learning performance
train_ds = train_ds.map(process)
# Prefetch training dataset to improve computational speed
train_ds = train_ds.cache().prefetch(buffer_size = autotune)

# Define validation dataset
val_ds = image_dataset_from_directory(
    validation_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize validation pixel values to improve learning performance
val_ds = val_ds.map(process)
# Prefetch validation dataset to improve computation speed
val_ds = val_ds.cache().prefetch(buffer_size = autotune)

# Compile model:
  # Adam algorithm optimization
  # Categorical crossentropy loss function for multilabel classification
  # Metrics: accuracy, precision, recall
res_net_50.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy', Precision(), Recall()]
)

# Define checkpoint to save model parameters to epoch with best performance
model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Models/ResNet-50/Original Training/Checkpoint', monitor = 'val_accuracy', mode = 'max')

# Train model to run until epochs finished or callback threshold reached
res_net_50.fit(
    train_ds,
    validation_data = val_ds,
    epochs = epochs,
    verbose = 1,
    callbacks = [model_checkpoint]
)

# Save ResNet-50 model at end of training
save_model(model = res_net_50, 
           filepath = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Models/ResNet-50/Original Training/End',
           include_optimizer = True,
)

# ----------------------------- Test ResNet-50 ---------------------------- #
print('\n')
# Define testing dataset
test_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Testing Dataset'
test_ds = image_dataset_from_directory(
    test_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = 1,
    label_mode = 'categorical'
)
# Normalize testing pixel values
test_ds = test_ds.map(process)

# Perform predictions on testing dataset to obtain accuracy, precision and recall
loss, accuracy, precision, recall = vgg16.evaluate(test_ds)
print('\nTest Accuracy: ' + str(accuracy))
print('Test Precision: ' + str(precision))
print('Test Recall: ' + str(recall))

94781440/94765736 [==============================] - 1s 0us/step
Found 2433 files belonging to 8 classes.
Found 695 files belonging to 8 classes.
Epoch 1/200
609/609 [==============================] - 691s 1s/step - loss: 2.0874 - accuracy: 0.1944 - precision: 0.3182 - recall: 0.0115 - val_loss: 1.9336 - val_accuracy: 0.1871 - val_precision: 0.4737 - val_recall: 0.0129
Epoch 2/200
609/609 [==============================] - 34s 55ms/step - loss: 1.9026 - accuracy: 0.2524 - precision: 0.7297 - recall: 0.0333 - val_loss: 1.8475 - val_accuracy: 0.2633 - val_precision: 0.6944 - val_recall: 0.0360
Epoch 3/200
609/609 [==============================] - 34s 56ms/step - loss: 1.8154 - accuracy: 0.2968 - precision: 0.7879 - recall: 0.0748 - val_loss: 1.7992 - val_accuracy: 0.2791 - val_precision: 0.7887 - val_recall: 0.0806
Epoch 4/200
609/609 [==============================] - 35s 57ms/step - loss: 1.7507 - accuracy: 0.3189 - precision: 0.8027 - recall: 0.0986 - val_loss: 1.7591 - val_accuracy:

*Augmented Dataset*

In [13]:
# Import os library to navigate directories
import os
# Import tensorflow functions for model building
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, Input, GlobalAveragePooling2D
from tensorflow.keras import Model
from tensorflow.keras.models import save_model
# Import tensorflow functions for training
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.data import AUTOTUNE as autotune
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import cast
# Import tensorflow functions for metrics
from tensorflow.keras.metrics import Accuracy, Precision, Recall

# ----------------------------- Build ResNet-50 ---------------------------- #
# Create base ResNet-50 model:
  # 224x224x3 input size
  # Only feature extraction layers - no classification layers
  # Feature extraction layer weights trained on ImageNet dataset
base_model = ResNet50(input_shape = (224, 224, 3),
                         include_top = False,
                         weights = 'imagenet'
                         )
# Freeze learning on feature extraction layers
base_model.trainable = False

# Create classification layers of ResNet-50
global_average_pool = GlobalAveragePooling2D()        # Global average pooling
fc1 = Dense(units = 1000, activation = 'relu')        # 1000 neuron fully connected layer with ReLU activation
classifier = Dense(units = 8, activation = 'softmax') # Softmax classifier

# Connect input layer, base model and output layers
inputs = Input(shape = (224, 224, 3)) # Input layer: 224x224 RGB images
model = base_model(inputs)            # Feature extraction layers of ResNet-50
model = global_average_pool(model)    # Global average pooling layer
model = fc1(model)                    # Fully connected layer: 1000 neuron ReLU
outputs = classifier(model)           # Fully connected layer: 8 neuron softmax activation for classification

# Build res_net_50 model
res_net_50 = Model(inputs, outputs, name = 'ResNet-50')

# ----------------------------- Train ResNet-50 ---------------------------- #
# RGB value normalization to improve learning rate - RGB values within activation function range
def process(image, label):
    image = cast(image/255.0, 'float32')
    return image, label

# Define training parameters
batch_size = 4   # Batch size for gradient learning
img_height = 224 # Image pixel height
img_width = 224  # Image pixel width
epochs = 200     # Arbitrarily large epoch for training

# Define training and validation directory paths
train_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Augmented Training Dataset'
validation_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Validation Dataset'

# Create training dataset
train_ds = image_dataset_from_directory(
    train_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize training dataset pixel values to improve learning performance
train_ds = train_ds.map(process)
# Prefetch training dataset to improve computational speed
train_ds = train_ds.cache().prefetch(buffer_size = autotune)

# Define validation dataset
val_ds = image_dataset_from_directory(
    validation_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize validation pixel values to improve learning performance
val_ds = val_ds.map(process)
# Prefetch validation dataset to improve computation speed
val_ds = val_ds.cache().prefetch(buffer_size = autotune)

# Compile model:
  # Adam algorithm optimization
  # Categorical crossentropy loss function for multilabel classification
  # Metrics: accuracy, precision, recall
res_net_50.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy', Precision(), Recall()]
)

# Define checkpoint to save model parameters to epoch with best performance
model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Models/ResNet-50/Augmented Training/Checkpoint', monitor = 'val_accuracy', mode = 'max')

# Train model to run until epochs finished or callback threshold reached
res_net_50.fit(
    train_ds,
    validation_data = val_ds,
    epochs = epochs,
    verbose = 1,
    callbacks = [model_checkpoint]
)

# Save ResNet-50 model at end of training
save_model(model = res_net_50, 
           filepath = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Models/ResNet-50/Augmented Training/End',
           include_optimizer = True,
)

# ----------------------------- Test ResNet-50 ---------------------------- #
print('\n')
# Define testing dataset
test_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Testing Dataset'
test_ds = image_dataset_from_directory(
    test_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = 1,
    label_mode = 'categorical'
)
# Normalize testing pixel values
test_ds = test_ds.map(process)

# Perform predictions on testing dataset to obtain accuracy, precision and recall
loss, accuracy, precision, recall = vgg16.evaluate(test_ds)
print('\nTest Accuracy: ' + str(accuracy))
print('Test Precision: ' + str(precision))
print('Test Recall: ' + str(recall))

Found 10083 files belonging to 8 classes.
Found 695 files belonging to 8 classes.
Epoch 1/200
   7/2521 [..............................] - ETA: 20:29 - loss: 2.8904 - accuracy: 0.1786 - precision_5: 0.0000e+00 - recall_5: 0.0000e+00

KeyboardInterrupt: ignored

# DenseNet121

*Original Dataset*

In [14]:
# Import os library to navigate directories
import os
# Import tensorflow functions for model building
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input
from tensorflow.keras import Model
from tensorflow.keras.models import save_model
# Import tensorflow functions for training
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.data import AUTOTUNE as autotune
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import cast
# Import tensorflow functions for metrics
from tensorflow.keras.metrics import Accuracy, Precision, Recall

# --------------------------- Build DenseNet121 ---------------------------- #
# Create base DenseNet121 model:
  # 224x224x3 input size
  # Only feature extraction layers - no classification layers
  # Feature extraction layer weights trained on ImageNet dataset
base_model = DenseNet121(input_shape = (224, 224, 3),
                         include_top = False,
                         weights = 'imagenet'
                         )
# Freeze learning on feature extraction layers
base_model.trainable = False

# Create classification layers of DenseNet121
# 7x7 global average pooling layer
global_average_pooling = GlobalAveragePooling2D()
# Fully connected layer using softmax activation function for 8 labels
classifier = Dense(units = 8, activation = 'softmax')

# Connect input layer, base model and output classifier
inputs = Input(shape = (224, 224, 3)) # Input layer: 224x224 RGB images
model = base_model(inputs)            # Feature extraction layers of DenseNet121
model = global_average_pooling(model) # Fully connected layer: 7x7 global average pooling
outputs = classifier(model)           # Fully connected layer: 8 neuron softmax activation for classification

# Build DenseNet121 model
dense_net_121 = Model(inputs, outputs, name = 'DenseNet121')

# --------------------------- Train DenseNet121 ---------------------------- #
# RGB value normalization to improve learning rate - RGB values within activation function range
def process(image, label):
    image = cast(image/255.0, 'float32')
    return image, label

# Define training parameters
batch_size = 4   # Batch size for gradient learning
img_height = 224 # Image pixel height
img_width = 224  # Image pixel width
epochs = 200     # Arbitrarily large epoch for training

# Define training and validation directory paths
train_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Original Training Dataset'
validation_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Validation Dataset'

# Create training dataset
train_ds = image_dataset_from_directory(
    train_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize training dataset pixel values to improve learning performance
train_ds = train_ds.map(process)
# Prefetch training dataset to improve computational speed
train_ds = train_ds.cache().prefetch(buffer_size = autotune)

# Define validation dataset
val_ds = image_dataset_from_directory(
    validation_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize validation pixel values to improve learning performance
val_ds = val_ds.map(process)
# Prefetch validation dataset to improve computation speed
val_ds = val_ds.cache().prefetch(buffer_size = autotune)

# Compile model:
  # Adam algorithm optimization
  # Categorical crossentropy loss function for multilabel classification
  # Metrics: accuracy, precision, recall
dense_net_121.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy', Precision(), Recall()]
)

# Define checkpoint to save model parameters to epoch with best performance
model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Models/DenseNet121/Original Training/Checkpoint', monitor = 'val_accuracy', mode = 'max')

# Train model to run until epochs finished or callback threshold reached
dense_net_121.fit(
    train_ds,
    validation_data = val_ds,
    epochs = epochs,
    verbose = 1,
    callbacks = [model_checkpoint]
)

# Save DenseNet121 model at end of training
save_model(model = dense_net_121, 
           filepath = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Models/DenseNet121/Original Training/End',
           include_optimizer = True,
)

# --------------------------- Test DenseNet121 ---------------------------- #
print('\n')
# Define testing dataset
test_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Testing Dataset'
test_ds = image_dataset_from_directory(
    test_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = 1,
    label_mode = 'categorical'
)
# Normalize testing pixel values
test_ds = test_ds.map(process)

# Perform predictions on testing dataset to obtain accuracy, precision and recall
loss, accuracy, precision, recall = vgg16.evaluate(test_ds)
print('\nTest Accuracy: ' + str(accuracy))
print('Test Precision: ' + str(precision))
print('Test Recall: ' + str(recall))

29097984/29084464 [==============================] - 1s 0us/step
Found 2433 files belonging to 8 classes.
Found 695 files belonging to 8 classes.
Epoch 1/200
  7/609 [..............................] - ETA: 4:07 - loss: 2.3962 - accuracy: 0.2857 - precision_6: 0.1667 - recall_6: 0.0357

KeyboardInterrupt: ignored

*Augmented Dataset*

In [15]:
# Import os library to navigate directories
import os
# Import tensorflow functions for model building
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input
from tensorflow.keras import Model
from tensorflow.keras.models import save_model
# Import tensorflow functions for training
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.data import AUTOTUNE as autotune
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import cast
# Import tensorflow functions for metrics
from tensorflow.keras.metrics import Accuracy, Precision, Recall

# --------------------------- Build DenseNet121 ---------------------------- #
# Create base DenseNet121 model:
  # 224x224x3 input size
  # Only feature extraction layers - no classification layers
  # Feature extraction layer weights trained on ImageNet dataset
base_model = DenseNet121(input_shape = (224, 224, 3),
                         include_top = False,
                         weights = 'imagenet'
                         )
# Freeze learning on feature extraction layers
base_model.trainable = False

# Create classification layers of DenseNet121
# 7x7 global average pooling layer
global_average_pooling = GlobalAveragePooling2D()
# Fully connected layer using softmax activation function for 8 labels
classifier = Dense(units = 8, activation = 'softmax')

# Connect input layer, base model and output classifier
inputs = Input(shape = (224, 224, 3)) # Input layer: 224x224 RGB images
model = base_model(inputs)            # Feature extraction layers of DenseNet121
model = global_average_pooling(model) # Fully connected layer: 7x7 global average pooling
outputs = classifier(model)           # Fully connected layer: 8 neuron softmax activation for classification

# Build DenseNet121 model
dense_net_121 = Model(inputs, outputs, name = 'DenseNet121')

# --------------------------- Train DenseNet121 ---------------------------- #
# RGB value normalization to improve learning rate - RGB values within activation function range
def process(image, label):
    image = cast(image/255.0, 'float32')
    return image, label

# Define training parameters
batch_size = 4   # Batch size for gradient learning
img_height = 224 # Image pixel height
img_width = 224  # Image pixel width
epochs = 200     # Training epochs

# Define training and validation directory paths
train_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Augmented Training Dataset'
validation_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Validation Dataset'

# Create training dataset
train_ds = image_dataset_from_directory(
    train_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize training dataset pixel values to improve learning performance
train_ds = train_ds.map(process)
# Prefetch training dataset to improve computational speed
train_ds = train_ds.cache().prefetch(buffer_size = autotune)

# Define validation dataset
val_ds = image_dataset_from_directory(
    validation_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize validation pixel values to improve learning performance
val_ds = val_ds.map(process)
# Prefetch validation dataset to improve computation speed
val_ds = val_ds.cache().prefetch(buffer_size = autotune)

# Compile model:
  # Adam algorithm optimization
  # Categorical crossentropy loss function for multilabel classification
  # Metrics: accuracy, precision, recall
dense_net_121.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy', Precision(), Recall()]
)

# Define checkpoint to save model parameters to epoch with best performance
model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Models/DenseNet121/Augmented Training/Checkpoint', monitor = 'val_accuracy', mode = 'max')

# Train model to run until epochs finished or callback threshold reached
dense_net_121.fit(
    train_ds,
    validation_data = val_ds,
    epochs = epochs,
    verbose = 1,
    callbacks = [model_checkpoint]
)

# Save DenseNet121 model at end of training
save_model(model = dense_net_121, 
           filepath = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Models/DenseNet121/Augmented Training/End',
           include_optimizer = True,
)

# --------------------------- Test DenseNet121 ---------------------------- #
# Define testing dataset
print('\n')
test_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Testing Dataset'
test_ds = image_dataset_from_directory(
    test_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = 1,
    label_mode = 'categorical'
)
# Normalize testing pixel values
test_ds = test_ds.map(process)

# Perform predictions on testing dataset to obtain accuracy, precision and recall
loss, accuracy, precision, recall = vgg16.evaluate(test_ds)
print('\nTest Accuracy: ' + str(accuracy))
print('Test Precision: ' + str(precision))
print('Test Recall: ' + str(recall))

Found 10083 files belonging to 8 classes.
Found 695 files belonging to 8 classes.
Epoch 1/200
   7/2521 [..............................] - ETA: 21:21 - loss: 2.9128 - accuracy: 0.1429 - precision_7: 0.0000e+00 - recall_7: 0.0000e+00

KeyboardInterrupt: ignored

# Inception V3

*Original Dataset*

In [16]:
# Import os library to navigate directories
import os
# Import tensorflow functions for model building
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense, Input, GlobalAveragePooling2D
from tensorflow.keras import Model
from tensorflow.keras.models import save_model
# Import tensorflow functions for training
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.data import AUTOTUNE as autotune
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import cast
# Import tensorflow functions for metrics
from tensorflow.keras.metrics import Accuracy, Precision, Recall

# ---------------------------- Build InceptionV3 --------------------------- #
# Create base InceptionV3 model:
  # 224x224x3 input size
  # Only feature extraction layers - no classification layers
  # Feature extraction layer weights trained on ImageNet dataset
base_model = InceptionV3(input_shape = (224, 224, 3),
                         include_top = False,
                         weights = 'imagenet'
                         )
# Freeze learning on feature extraction layers
base_model.trainable = False

# Create classification layers of InceptionV3
global_average_pool = GlobalAveragePooling2D()        # global average pooling
fc1 = Dense(units = 2048, activation = 'relu')        # 1000 neuron fully connected layer with ReLU activation
classifier = Dense(units = 8, activation = 'softmax') # Softmax classifier

# Connect input layer, base model and output layers
inputs = Input(shape = (224, 224, 3)) # Input layer: 224x224 RGB images
model = base_model(inputs)            # Feature extraction layers of InceptionV3
model = global_average_pool(model)    # Global average pooling layer
model = fc1(model)                    # Fully connected layer: 2048 neuron ReLU
outputs = classifier(model)           # Fully connected layer: 8 neuron softmax activation for classification

# Build inception_v3 model
inception_v3 = Model(inputs, outputs, name = 'InceptionV3')

# ---------------------------- Train InceptionV3 -------------------------- #
# RGB value normalization to improve learning rate - RGB values within activation function range
def process(image, label):
    image = cast(image/255.0, 'float32')
    return image, label

# Define training parameters
batch_size = 4   # Batch size for gradient learning
img_height = 224 # Image pixel height
img_width = 224  # Image pixel width
epochs = 200     # Arbitrarily large epoch for training

# Define training and validation directory paths
train_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Original Training Dataset'
validation_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Validation Dataset'

# Create training dataset
train_ds = image_dataset_from_directory(
    train_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize training dataset pixel values to improve learning performance
train_ds = train_ds.map(process)
# Prefetch training dataset to improve computational speed
train_ds = train_ds.cache().prefetch(buffer_size = autotune)

# Define validation dataset
val_ds = image_dataset_from_directory(
    validation_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize validation pixel values to improve learning performance
val_ds = val_ds.map(process)
# Prefetch validation dataset to improve computation speed
val_ds = val_ds.cache().prefetch(buffer_size = autotune)

# Compile model:
  # Adam algorithm optimization
  # Categorical crossentropy loss function for multilabel classification
  # Metrics: accuracy, precision, recall
inception_v3.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy', Precision(), Recall()]
)

# Define checkpoint to save model parameters to epoch with best performance
model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Models/InceptionV3/Original Training/Checkpoint', monitor = 'val_accuracy', mode = 'max')

# Train model to run until epochs finished or callback threshold reached
inception_v3.fit(
    train_ds,
    validation_data = val_ds,
    epochs = epochs,
    verbose = 1,
    callbacks = [model_checkpoint]
)

# Save Inception V3 model at end of training
save_model(model = inception_v3, 
           filepath = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Models/InceptionV3/Original Training/End',
           include_optimizer = True,
)

# ---------------------------- Test InceptionV3 -------------------------- #
# Define testing dataset
print('\n')
test_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Testing Dataset'
test_ds = image_dataset_from_directory(
    test_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = 1,
    label_mode = 'categorical'
)
# Normalize testing pixel values
test_ds = test_ds.map(process)

# Perform predictions on testing dataset to obtain accuracy, precision and recall
loss, accuracy, precision, recall = vgg16.evaluate(test_ds)
print('\nTest Accuracy: ' + str(accuracy))
print('Test Precision: ' + str(precision))
print('Test Recall: ' + str(recall))

87924736/87910968 [==============================] - 1s 0us/step
Found 2433 files belonging to 8 classes.
Found 695 files belonging to 8 classes.
Epoch 1/200
 16/609 [..............................] - ETA: 3:23 - loss: 6.2140 - accuracy: 0.2031 - precision_8: 0.2400 - recall_8: 0.1875

KeyboardInterrupt: ignored

*Augmented Dataset*

In [18]:
# Import os library to navigate directories
import os
# Import tensorflow functions for model building
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense, Input, GlobalAveragePooling2D
from tensorflow.keras import Model
from tensorflow.keras.models import save_model
# Import tensorflow functions for training
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.data import AUTOTUNE as autotune
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import cast
# Import tensorflow functions for metrics
from tensorflow.keras.metrics import Accuracy, Precision, Recall

# ---------------------------- Build InceptionV3 --------------------------- #
# Create base InceptionV3 model:
  # 224x224x3 input size
  # Only feature extraction layers - no classification layers
  # Feature extraction layer weights trained on ImageNet dataset
base_model = InceptionV3(input_shape = (224, 224, 3),
                         include_top = False,
                         weights = 'imagenet'
                         )
# Freeze learning on feature extraction layers
base_model.trainable = False

# Create classification layers of InceptionV3
global_average_pool = GlobalAveragePooling2D()        # global average pooling
fc1 = Dense(units = 2048, activation = 'relu')        # 1000 neuron fully connected layer with ReLU activation
classifier = Dense(units = 8, activation = 'softmax') # Softmax classifier

# Connect input layer, base model and output layers
inputs = Input(shape = (224, 224, 3)) # Input layer: 224x224 RGB images
model = base_model(inputs)            # Feature extraction layers of InceptionV3
model = global_average_pool(model)    # Global average pooling layer
model = fc1(model)                    # Fully connected layer: 2048 neuron ReLU
outputs = classifier(model)           # Fully connected layer: 8 neuron softmax activation for classification

# Build inception_v3 model
inception_v3 = Model(inputs, outputs, name = 'InceptionV3')

# ---------------------------- Train InceptionV3 -------------------------- #
# RGB value normalization to improve learning rate - RGB values within activation function range
def process(image, label):
    image = cast(image/255.0, 'float32')
    return image, label

# Define training parameters
batch_size = 4   # Batch size for gradient learning
img_height = 224 # Image pixel height
img_width = 224  # Image pixel width
epochs = 200     # Arbitrarily large epoch for training

# Define training and validation directory paths
train_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Augmented Training Dataset'
validation_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Validation Dataset'

# Create training dataset
train_ds = image_dataset_from_directory(
    train_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize training dataset pixel values to improve learning performance
train_ds = train_ds.map(process)
# Prefetch training dataset to improve computational speed
train_ds = train_ds.cache().prefetch(buffer_size = autotune)

# Define validation dataset
val_ds = image_dataset_from_directory(
    validation_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize validation pixel values to improve learning performance
val_ds = val_ds.map(process)
# Prefetch validation dataset to improve computation speed
val_ds = val_ds.cache().prefetch(buffer_size = autotune)

# Compile model:
  # Adam algorithm optimization
  # Categorical crossentropy loss function for multilabel classification
  # Metrics: accuracy, precision, recall
inception_v3.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy', Precision(), Recall()]
)

# Define checkpoint to save model parameters to epoch with best performance
model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Models/InceptionV3/Augmented Training/Checkpoint', monitor = 'val_accuracy', mode = 'max')

# Train model to run until epochs finished or callback threshold reached
inception_v3.fit(
    train_ds,
    validation_data = val_ds,
    epochs = epochs,
    verbose = 1,
    callbacks = [model_checkpoint]
)

# Save Inception V3 model at end of training
save_model(model = inception_v3, 
           filepath = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Models/InceptionV3/Augmented Training/End',
           include_optimizer = True,
)

# ---------------------------- Test InceptionV3 -------------------------- #
# Define testing dataset
print('\n')
test_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Testing Dataset'
test_ds = image_dataset_from_directory(
    test_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = 1,
    label_mode = 'categorical'
)
# Normalize testing pixel values
test_ds = test_ds.map(process)

# Perform predictions on testing dataset to obtain accuracy, precision and recall
loss, accuracy, precision, recall = vgg16.evaluate(test_ds)
print('\nTest Accuracy: ' + str(accuracy))
print('Test Precision: ' + str(precision))
print('Test Recall: ' + str(recall))

Found 10083 files belonging to 8 classes.
Found 695 files belonging to 8 classes.
Epoch 1/200
   7/2521 [..............................] - ETA: 13:48 - loss: 5.4527 - accuracy: 0.2143 - precision_9: 0.2222 - recall_9: 0.1429

KeyboardInterrupt: ignored

# MobileNet V2

*Original Dataset*

In [19]:
# Import os library to navigate directories
import os
# Import tensorflow functions for model building
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.layers import Dense, Input, GlobalAveragePooling2D
from tensorflow.keras import Model
from tensorflow.keras.models import save_model
# Import tensorflow functions for training
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.data import AUTOTUNE as autotune
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import cast
# Import tensorflow functions for metrics
from tensorflow.keras.metrics import Accuracy, Precision, Recall

# ---------------------------- Build MobileNetV2 --------------------------- #
# Create base MobileNetV2 model:
  # 224x224x3 input size
  # Only feature extraction layers - no classification layers
  # Feature extraction layer weights trained on ImageNet dataset
base_model = MobileNetV2(input_shape = (224, 224, 3),
                         include_top = False,
                         weights = 'imagenet'
                         )
# Freeze learning on feature extraction layers
base_model.trainable = False

# Create classification layers of InceptionV3
global_average_pool = GlobalAveragePooling2D()        # global average pooling
classifier = Dense(units = 8, activation = 'softmax') # 1000 neuron fully connected layer with ReLU activation

# Connect input layer, base model and output layers
inputs = Input(shape = (224, 224, 3)) # Input layer: 224x224 RGB images
model = base_model(inputs)            # Feature extraction layers of InceptionV3
model = global_average_pool(model)    # Global average pooling layer
outputs = classifier(model)           # Fully connected layer: 8 neuron softmax activation for classification

# Build inception_v3 model
mobile_net_v2 = Model(inputs, outputs, name = 'MobileNetV2')

# ---------------------------- Train MobileNetV2 -------------------------- #
# RGB value normalization to improve learning rate - RGB values within activation function range
def process(image, label):
    image = cast(image/255.0, 'float32')
    return image, label

# Define training parameters
batch_size = 4   # Batch size for gradient learning
img_height = 224 # Image pixel height
img_width = 224  # Image pixel width
epochs = 200     # Arbitrarily large epoch for training

# Define training and validation directory paths
train_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Original Training Dataset'
validation_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Validation Dataset'

# Create training dataset
train_ds = image_dataset_from_directory(
    train_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize training dataset pixel values to improve learning performance
train_ds = train_ds.map(process)
# Prefetch training dataset to improve computational speed
train_ds = train_ds.cache().prefetch(buffer_size = autotune)

# Define validation dataset
val_ds = image_dataset_from_directory(
    validation_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize validation pixel values to improve learning performance
val_ds = val_ds.map(process)
# Prefetch validation dataset to improve computation speed
val_ds = val_ds.cache().prefetch(buffer_size = autotune)

# Compile model:
  # Adam algorithm optimization
  # Categorical crossentropy loss function for multilabel classification
  # Metrics: accuracy, precision, recall
mobile_net_v2.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy', Precision(), Recall()]
)

# Define checkpoint to save model parameters to epoch with best performance
model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Models/MobileNetV2/Original Training/Checkpoint', monitor = 'val_accuracy', mode = 'max')

# Train model to run until epochs finished or callback threshold reached
mobile_net_v2.fit(
    train_ds,
    validation_data = val_ds,
    epochs = epochs,
    verbose = 1,
    callbacks = [model_checkpoint]
)

# Save MobileNetV2 model at end of training
save_model(model = mobile_net_v2, 
           filepath = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Models/MobileNetV2/Original Training/End',
           include_optimizer = True,
)

# ---------------------------- Test MobileNetV2 -------------------------- #
# Define testing dataset
print('\n')
test_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Testing Dataset'
test_ds = image_dataset_from_directory(
    test_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = 1,
    label_mode = 'categorical'
)
# Normalize testing pixel values
test_ds = test_ds.map(process)

# Perform predictions on testing dataset to obtain accuracy, precision and recall
loss, accuracy, precision, recall = vgg16.evaluate(test_ds)
print('\nTest Accuracy: ' + str(accuracy))
print('Test Precision: ' + str(precision))
print('Test Recall: ' + str(recall))

9420800/9406464 [==============================] - 0s 0us/step
Found 2433 files belonging to 8 classes.
Found 695 files belonging to 8 classes.
Epoch 1/200
 40/609 [>.............................] - ETA: 51s - loss: 1.9837 - accuracy: 0.3063 - precision_10: 0.4500 - recall_10: 0.1125

KeyboardInterrupt: ignored

*Augmented Dataset*

In [20]:
# Import os library to navigate directories
import os
# Import tensorflow functions for model building
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.layers import Dense, Input, GlobalAveragePooling2D
from tensorflow.keras import Model
from tensorflow.keras.models import save_model
# Import tensorflow functions for training
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.data import AUTOTUNE as autotune
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import cast
# Import tensorflow functions for metrics
from tensorflow.keras.metrics import Accuracy, Precision, Recall

# ---------------------------- Build MobileNetV2 --------------------------- #
# Create base MobileNetV2 model:
  # 224x224x3 input size
  # Only feature extraction layers - no classification layers
  # Feature extraction layer weights trained on ImageNet dataset
base_model = MobileNetV2(input_shape = (224, 224, 3),
                         include_top = False,
                         weights = 'imagenet'
                         )
# Freeze learning on feature extraction layers
base_model.trainable = False

# Create classification layers of InceptionV3
global_average_pool = GlobalAveragePooling2D()        # global average pooling
classifier = Dense(units = 8, activation = 'softmax') # 1000 neuron fully connected layer with ReLU activation

# Connect input layer, base model and output layers
inputs = Input(shape = (224, 224, 3)) # Input layer: 224x224 RGB images
model = base_model(inputs)            # Feature extraction layers of InceptionV3
model = global_average_pool(model)    # Global average pooling layer
outputs = classifier(model)           # Fully connected layer: 8 neuron softmax activation for classification

# Build inception_v3 model
mobile_net_v2 = Model(inputs, outputs, name = 'MobileNetV2')

# ---------------------------- Train MobileNetV2 -------------------------- #
# RGB value normalization to improve learning rate - RGB values within activation function range
def process(image, label):
    image = cast(image/255.0, 'float32')
    return image, label

# Define training parameters
batch_size = 4   # Batch size for gradient learning
img_height = 224 # Image pixel height
img_width = 224  # Image pixel width
epochs = 200     # Arbitrarily large epoch for training

# Define training and validation directory paths
train_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Augmented Training Dataset'
validation_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Validation Dataset'

# Create training dataset
train_ds = image_dataset_from_directory(
    train_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize training dataset pixel values to improve learning performance
train_ds = train_ds.map(process)
# Prefetch training dataset to improve computational speed
train_ds = train_ds.cache().prefetch(buffer_size = autotune)

# Define validation dataset
val_ds = image_dataset_from_directory(
    validation_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize validation pixel values to improve learning performance
val_ds = val_ds.map(process)
# Prefetch validation dataset to improve computation speed
val_ds = val_ds.cache().prefetch(buffer_size = autotune)

# Compile model:
  # Adam algorithm optimization
  # Categorical crossentropy loss function for multilabel classification
  # Metrics: accuracy, precision, recall
mobile_net_v2.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy', Precision(), Recall()]
)

# Define checkpoint to save model parameters to epoch with best performance
model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Models/MobileNetV2/Augmented Training/Checkpoint', monitor = 'val_accuracy', mode = 'max')

# Train model to run until epochs finished or callback threshold reached
mobile_net_v2.fit(
    train_ds,
    validation_data = val_ds,
    epochs = epochs,
    verbose = 1,
    callbacks = [model_checkpoint]
)

# Save MobileNetV2 model at end of training
save_model(model = mobile_net_v2, 
           filepath = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Models/MobileNetV2/Augmented Training/End',
           include_optimizer = True,
)

# ---------------------------- Test MobileNetV2 -------------------------- #
# Define testing dataset
print('\n')
test_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Testing Dataset'
test_ds = image_dataset_from_directory(
    test_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = 1,
    label_mode = 'categorical'
)
# Normalize testing pixel values
test_ds = test_ds.map(process)

# Perform predictions on testing dataset to obtain accuracy, precision and recall
loss, accuracy, precision, recall = vgg16.evaluate(test_ds)
print('\nTest Accuracy: ' + str(accuracy))
print('Test Precision: ' + str(precision))
print('Test Recall: ' + str(recall))

Found 10083 files belonging to 8 classes.
Found 695 files belonging to 8 classes.
Epoch 1/200
  24/2521 [..............................] - ETA: 13:40 - loss: 2.3511 - accuracy: 0.2292 - precision_11: 0.2222 - recall_11: 0.0417

KeyboardInterrupt: ignored

# Model Testing

In [21]:
# Import directory navigation
from os import listdir
# Import tensorflow libraries for loading models and testing dataset
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow import cast
# Import tensorflow functions for metrics
from tensorflow.keras.metrics import Accuracy, Precision, Recall
#from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np

# Constants for model file paths
base_model_path = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Models/'
models = ['VGG-16', 'ResNet-50', 'DenseNet121', 'InceptionV3', 'MobileNetV2']

# ------------------------- Dataset and Testing Functions ------------------------ #
def process(image, label):
    image = cast(image/255.0, 'float32')
    return image, label

def test_model(test_ds, model_path):
    # Evaluate model on test dataset for accuracy, precision and recall
    model = load_model(model_path)
    #loss, accuracy, precision, recall = model.evaluate(test_ds)
    print('Accuracy: ' + accuracy)
    print('Precision: ' + precision)
    print('Recall: ' + recall)
    # Evaluate model for confusion matrix #########
    #predictions = model.predict(test_ds)
    #cm = confusion_matrix(test_ds, np.rint(predictions), labels = ['Glass', 'Plastic', 'Paper', 'Metal', 'Cardboard', 'Vegetation', 'Food', 'Trash'])
    #display = ConfusionMatrixDisplay(cm)
    #display.plot()
    

# ----------------------------- Load Testing Dataset ----------------------------- #
# Define testing dataset
test_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Testing Dataset'
test_ds = image_dataset_from_directory(
    test_dir,
    validation_split = 0,
    seed = 123,
    image_size = (224, 224),
    batch_size = 346,
    label_mode = 'categorical'
)
# Normalize testing pixel values
test_ds = test_ds.map(process)

# ----------------------------- Load and Test Models ----------------------------- #
for model in models:
    print('\n# ---------------------------------------- '+model+' ----------------------------------------\n')
    # Original model fully trained
    print('Original Dataset Training - Full Training:')
    test_model(test_ds, base_model_path+model+'/Original Training/End')
    print('\n')
    # Original model callback to best validation performance
    print('Original Dataset Training - Callback to Best Validation Performance:')
    test_model(test_ds, base_model_path+model+'/Original Training/Checkpoint')
    print('\n')
    # Augmented training dataset model fully trained
    print('Augmented Dataset Training - Full Training:')
    test_model(test_ds, base_model_path+model+'/Augmented Training/End')
    print('\n')
    # Augmented training dataset model callback to best validation performance
    print('Augmented Dataset Training - Callback to Best Validation Performance:')
    test_model(test_ds, base_model_path+model+'/Augmented Training/Checkpoint')

Found 346 files belonging to 8 classes.

# ---------------------------------------- VGG-16 ----------------------------------------

Original Dataset Training - Full Training:


KeyboardInterrupt: ignored

# Data Augmentation and Preprocessing

In [ ]:
!pip install Augmentor

*Dataset Split and Preprocessing: 70:20:10 (training, validation, testing) split*

In [ ]:
# Import directory navigation libraries and image resizing
from os import listdir
from shutil import move
from random import randint
from PIL import Image
from tensorflow.image import resize
from tensorflow.io import decode_jpeg, encode_jpeg, write_file
from tensorflow.dtypes import saturate_cast

# Constant values
base_path = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/'
base_validation_path = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Validation Dataset/'
base_test_path = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Testing Dataset/'
base_original_training_path = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Original Training Dataset/'
labels = ['Glass', 'Plastic', 'Paper', 'Metal', 'Cardboard', 'Vegetation', 'Food', 'Trash']

# ------------------------------- Functions -------------------------------- #
# Resize images
def resize_image(path, img_width, img_height):
    directory = listdir(path)
    for file in directory:
        image = open(path+'/'+file, 'rb') # Open image as binary file
        binary_representation = image.read() # Read binary file
        decoded_representation = decode_jpeg(binary_representation) # Decode image
        resized_image = resize(decoded_representation, [img_width, img_height])
        resized_image = saturate_cast(resized_image, 'uint8')
        encoded_image = encode_jpeg(resized_image)
        write_file(path+'/'+file, encoded_image)

# Check images for RGB mode and convert if required
def convert_rgb_images(path):
    directory = listdir(path)
    for file in directory:
        img = Image.open(path+'/'+file)
        if img.mode != 'RGB':
            img.convert('RGB').save(path+'/'+file)
        
# Count number of files in directory
def count_files(inputPath, label):
    directory = listdir(inputPath+label)
    num_files = 0
    for file in directory:
        num_files += 1
    return num_files

# Split training and testing dataset
def split_dataset(input_path, train_path, val_path, test_path, val_prop, test_prop, label):
    # Count images in directory and get files names
    num_files = count_files(input_path, label)
    file_names = listdir(input_path+label)

    # Calculated number of validation and testing images
    num_val = round(num_files*val_prop)
    num_test = round(num_files*test_prop)
    
    # Initialise empty lists for validation and testing image indexes
    val_list = []
    test_list = []
    
    # Randomly assign file indexes for validation dataset
    i = 0
    while i < num_val:
        r = randint(1, num_files - 1)
        if r not in val_list:
            val_list.append(r)
            i += 1
    # Move validation dataset files and rename
    name_incrementer = 1
    for j in val_list:
        move(input_path+label+'/'+file_names[j], val_path+label+'/'+label+'_'+str(name_incrementer)+'.jpg')
        name_incrementer += 1

    # Update number of files and directory for remaining files
    num_files = count_files(input_path, label)
    remaining_files = listdir(input_path+label)
    # Randomly assign file indexes for testing dataset
    i = 0
    while i < num_test:
        r = randint(1, num_files - 1)
        if r not in test_list:
            test_list.append(r)
            i += 1
    # Move test dataset files and rename
    name_incrementer = 1
    for j in test_list:
        move(input_path+label+'/'+remaining_files[j], test_path+label+'/'+label+'_'+str(name_incrementer)+'.jpg')
        name_incrementer += 1

    # Move remaining files into training dataset
    training_files = listdir(input_path+label)
    name_incrementer = 1
    for file in training_files:
        move(input_path+label+'/'+file, train_path+label+'/'+label+'_'+str(name_incrementer)+'.jpg')
        name_incrementer += 1

# Print number of files in label directories
def print_num_files(path, label):
    directory = listdir(path+label)
    counter = 0
    for file in directory:
        counter += 1
    print("Number of files in " + label +':' + str(counter))

# ----------------- Split Training and Validation Datasets ----------------- #
# Print split label contents
print('# --------------- Total Label Counts ---------------- #')
# Split labels base off 70:20:10 training split
for label in labels:
    print_num_files(base_path+'Unsplit Dataset/', label)
    split_dataset(base_path+'Unsplit Dataset/', base_original_training_path, base_validation_path, base_test_path, 0.2, 0.1, label)

# Print validation counts for each label
print('\n# ---------- Total Validation Label Counts ---------- #')
for label in labels:
    print_num_files(base_validation_path, label)

# Print testing counts for each label
print('\n# ------------ Total Testing Label Counts ----------- #')
for label in labels:
    print_num_files(base_test_path, label)

# Print training counts for each label in training dataset
print('\n# ----------- Total Training Label Counts ----------- #')
for label in labels:
    print_num_files(base_original_training_path, label)

# ------------------------- Pre-Process Images -------------------------- #
# Pre-process training dataset
print('\n# --------- Pre-Processing Training Dataset --------- #')
for label in labels:
    convert_rgb_images(base_original_training_path+label)
    resize_image(base_original_training_path+label, 224, 224)
# Pre-process validation dataset
print('\n# -------- Pre-Processing Validation Dataset -------- #')
for label in labels:
    convert_rgb_images(base_original_training_path+label)
    resize_image(base_validation_path+label, 224, 224)
# Pre-process test dataset
print('\n# --------- Pre-Processing Testing Dataset --------- #')
for label in labels:
    convert_rgb_images(base_original_training_path+label)
    resize_image(base_test_path+label, 224, 224)

*Augementation Set 1: Distort and Flip*

In [ ]:
# Import data augmentation functions
import Augmentor
from os import listdir
from time import sleep

# Define base path of training dataset
base_original_training_path = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Original Training Dataset/'
# Define labels
#labels = ['Glass', 'Plastic', 'Paper', 'Metal', 'Cardboard', 'Vegetation', 'Food', 'Trash']
labels = ['Glass']

# Verify augmentation counts and print
def count_augmented_and_print(path):
    count_augmented = 0
    directory = listdir(path)
    for file in directory:
        count_augmented += 1
    print('Processed with '+str(count_augmented)+' image(s) found.\n')

# Geometric augmentation function: distort and flip
def distort_and_flip(path):
    pipe = Augmentor.Pipeline(path)           # Create pipeline in path of dataset
    pipe.flip_left_right(1)                   # Horizontally flip every image
    pipe.random_distortion(1, 5, 5, 4)        # Randomly distort flipped images
    pipe.process()                            # Process images in pipeline
    count_augmented_and_print(path+'/output') # Print augmented image count to console
    sleep(60)                                 # Sleep program while images processing

# Perform augmentation
for label in labels:
    distort_and_flip(base_original_training_path+label)

*Augmentation Set 2: Rotate and Shear*

In [ ]:
# Import data augmentation functions
import Augmentor
from os import listdir
from time import sleep

# Define base path of training dataset
base_original_training_path = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Original Training Dataset/'
# Define labels
labels = ['Glass', 'Plastic', 'Paper', 'Metal', 'Cardboard', 'Vegetation', 'Food', 'Trash']

# Verify augmentation counts and print
def count_augmented_and_print(path):
    count_augmented = 0
    directory = listdir(path)
    for file in directory:
        count_augmented += 1
    print('Processed with '+str(count_augmented)+' image(s) found.\n')

# Geometric augmentation: rotate and shear
def rotate_and_shear(path):
    pipe = Augmentor.Pipeline(path)           # Create pipeline in path of dataset
    pipe.rotate(1, 25, 25)                    # Rotate every image by 45 degrees
    pipe.shear(1, 15, 15)                     # Randomly shear between 0 and 15 degrees
    pipe.process()                            # Process images in pipeline
    count_augmented_and_print(path+'/output') # Print augmented image count to console
    sleep(60)                                 # Sleep program while images processing

# Augment images
for label in labels:
    rotate_and_shear(base_original_training_path+label)

*Augmentation Set 3: Random Erasing*

In [ ]:
# Import data augmentation functions
import Augmentor
from os import listdir
from time import sleep

# Define base path of training dataset
base_original_training_path = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Original Training Dataset/'
# Define labels
labels = ['Glass', 'Plastic', 'Paper', 'Metal', 'Cardboard', 'Vegetation', 'Food', 'Trash']

# Verify augmentation counts and print
def count_augmented_and_print(path):
    count_augmented = 0
    directory = listdir(path)
    for file in directory:
        count_augmented += 1
    print('Processed with '+str(count_augmented)+' image(s) found.\n')

# Random occlusion: random erasing 
def random_erase(path):
    pipe = Augmentor.Pipeline(path)           # Create pipeline in path of dataset
    pipe.random_erasing(1, 0.15)              # Randomly erase area in every image
    pipe.process()                            # Process images in pipeline
    count_augmented_and_print(path+'/output') # Print augmented image count to console
    sleep(60)                                 # Sleep program while images processing

# Augment images
for label in labels:
    random_erase(base_original_training_path+label)

# Second augmentation
for label in labels:
    random_erase(base_original_training_path+label+'/output')

*Rename Files*

In [ ]:
# Import libraries
from os import listdir
from shutil import move

# Constants
set_one_path = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/1 Distort and Flip/'
set_two_path = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/2 Rotate and Shear/'
set_three_path = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/3 Random Erasing/'
set_one_augmentation = 'distort_and_flip'
set_two_augmentation = 'shear_and_rotate'
set_three_augmentation = 'random_erase'
labels = ['Glass', 'Plastic', 'Paper', 'Metal', 'Cardboard', 'Vegetation', 'Food', 'Trash']

# Get files in directory and rename
def rename_files(path, base_name, augmentation):
    name_incrementer = 1
    directory = listdir(path)
    for file in directory:
        move(path+'/'+file, path+'/'+base_name+'_'+augmentation+'_'+str(name_incrementer)+'.jpg')
        name_incrementer += 1
        print(file)
    return name_incrementer

# Rename files
for label in labels:
    rename_files(set_one_path+label+'/output', label, set_one_augmentation)
    rename_files(set_two_path+label+'/output', label, set_two_augmentation)
    rename_files(set_three_path+label+'/output', label, set_three_augmentation)

*File Counts*

In [ ]:
# Import file navigation library
from os import listdir

# Directory paths
base_validation_path = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Validation Dataset/'
base_test_path = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Testing Dataset/'
base_original_training_path = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Original Training Dataset/'
base_augmented_training_path = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Augmented Training Dataset/'

# Print number of files in label directories
def print_num_files(path, label):
    directory = listdir(path+label)
    counter = 0
    for file in directory:
        counter += 1
    print("Number of files in " + label +':' + str(counter))

# Print training counts for each label in training dataset
print('\n# ----------- Total Training Label Counts ----------- #')
for label in labels:
    print_num_files(base_original_training_path, label)

# Print validation counts for each label
print('\n# ---------- Total Validation Label Counts ---------- #')
for label in labels:
    print_num_files(base_validation_path, label)

# Print testing counts for each label
print('\n# ------------ Total Testing Label Counts ----------- #')
for label in labels:
    print_num_files(base_test_path, label)

# Print training counts for each label in training dataset
print('\n# ----------- Total Training Label Counts ----------- #')
for label in labels:
    print_num_files(base_original_training_path, label)

# Print augmented counts for each label in augmented training dataset
print('\n# ----------- Total Augmented Label Counts ----------- #')
for label in labels:
    print_num_files(base_original_training_path, label)

In [ ]:
# Import data augmentation functions
import Augmentor

# Define path of sample image for each augmentation
base_path_set_one = '/content/drive/MyDrive/Colab Notebooks/Data Augmentation Samples/1 Distort and Flip'
base_path_set_two = '/content/drive/MyDrive/Colab Notebooks/Data Augmentation Samples/2 Rotate and Shear'
base_path_set_three = '/content/drive/MyDrive/Colab Notebooks/Data Augmentation Samples/3 Random Erasing'

# Geometric augmentation function: distort and flip
def distort_and_flip(path):
    pipe = Augmentor.Pipeline(path)           # Create pipeline in path of dataset
    pipe.flip_left_right(1)                   # Horizontally flip every image
    pipe.random_distortion(1, 5, 5, 4)        # Randomly distort flipped images
    pipe.process()                            # Process images in pipeline

# Geometric augmentation: rotate and shear
def rotate_and_shear(path):
    pipe = Augmentor.Pipeline(path)           # Create pipeline in path of dataset
    pipe.rotate(1, 25, 25)                    # Rotate every image by 45 degrees
    pipe.shear(1, 15, 15)                     # Randomly shear between 0 and 15 degrees
    pipe.process()                            # Process images in pipeline

# Random occlusion: random erasing 
def random_erase(path):
    pipe = Augmentor.Pipeline(path)           # Create pipeline in path of dataset
    pipe.random_erasing(1, 0.15)              # Randomly erase area in every image
    pipe.process()                            # Process images in pipeline

# Perform augmentations
# Distort and flip
distort_and_flip(base_path_set_one)
# Rotate and shear
rotate_and_shear(base_path_set_two)
# Random erase
random_erase(base_path_set_three)
random_erase(base_path_set_three+'/output')